# Benchmark for CPU vs GPU (For Machine Learning):

I decided to see the performance difference between training on a CPU vs GPU.

I used code from https://www.analyticsvidhya.com/blog/2021/11/benchmarking-cpu-and-gpu-performance-with-tensorflow/ , and will put in any notes that I make from this small experiment.

Importing the necessary libraries

In [20]:
#Importing libraries into the session:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

Testing if TensorFlow is running on the GPU, by listing all the physical devices that can be used:

In [21]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Testing if I am using CUDA on the GPU (using a GTX 950 in this test)

In [22]:
tf.test.is_built_with_cuda()

True

Loading in a large dataset (This is a keras dataset).

This uses the ***famous*** cifar10 dataset, which includes 32*32 colour images, of which is seperated into 50,000 train and 10,000 test images.

In [23]:
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

Checking the image shape. Also checking the data to see if it is as expected.

In [24]:
# checking images shape
X_train.shape, X_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

Making sure the database is working by querying the first entry

In [25]:
# display single image shape
X_train[0].shape

(32, 32, 3)

Making sure that the labels are as expected

In [26]:
# checking labels
y_train[:5]

array([[6],
       [9],
       [9],
       [4],
       [1]], dtype=uint8)

In [27]:
# scaling image values between 0-1
X_train_scaled = X_train/255
X_test_scaled = X_test/255

In [28]:
# one hot encoding labels
y_train_encoded = keras.utils.to_categorical(y_train, num_classes = 10, dtype = 'float32')
y_test_encoded = keras.utils.to_categorical(y_test, num_classes = 10, dtype = 'float32')

Using a function to build a neural netowrk with architecture. Compiling is also included.

In [29]:
def get_model():
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(32,32,3)),
        keras.layers.Dense(3000, activation='relu'),
        keras.layers.Dense(1000, activation='relu'),
        keras.layers.Dense(10, activation='sigmoid')    
    ])
    model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model


Testing the CPU in a 10 epoch run (This was run on a i7-2600)

Looking at telemetry, the CPU was pinned the entire time it was training (100%), and the system was hardly usable.

In [30]:
%%timeit -n1 -r1
# CPU
with tf.device('/CPU:0'):
    model_cpu = get_model()
    model_cpu.fit(X_train_scaled, y_train_encoded, epochs = 10)

Epoch 1/10
1563/1563 [==============================] - 141s 90ms/step - loss: 1.8110 - accuracy: 0.3555
Epoch 2/10
1563/1563 [==============================] - 136s 87ms/step - loss: 1.6217 - accuracy: 0.4274
Epoch 3/10
1563/1563 [==============================] - 137s 88ms/step - loss: 1.5401 - accuracy: 0.4568
Epoch 4/10
1563/1563 [==============================] - 140s 89ms/step - loss: 1.4799 - accuracy: 0.4783
Epoch 5/10
1563/1563 [==============================] - 141s 90ms/step - loss: 1.4301 - accuracy: 0.4952
Epoch 6/10
1563/1563 [==============================] - 138s 88ms/step - loss: 1.3863 - accuracy: 0.5129
Epoch 7/10
1563/1563 [==============================] - 140s 89ms/step - loss: 1.3504 - accuracy: 0.5257
Epoch 8/10
1563/1563 [==============================] - 143s 91ms/step - loss: 1.3140 - accuracy: 0.5358
Epoch 9/10
1563/1563 [==============================] - 142s 91ms/step - loss: 1.2822 - accuracy: 0.5497
Epoch 10/10
1563/1563 [==============================] 

Testing the GPU in a 10 epoch run (This was run on a GTX 950)

In [31]:
%%timeit -n1 -r1
# GPU
with tf.device('/GPU:0'):
    model_gpu = get_model()
    model_gpu.fit(X_train_scaled, y_train_encoded, epochs = 10)

Epoch 1/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.8144 - accuracy: 0.3561
Epoch 2/10
1563/1563 [==============================] - 10s 7ms/step - loss: 1.6277 - accuracy: 0.4267
Epoch 3/10
1563/1563 [==============================] - 10s 7ms/step - loss: 1.5440 - accuracy: 0.4557
Epoch 4/10
1563/1563 [==============================] - 10s 6ms/step - loss: 1.4846 - accuracy: 0.4777
Epoch 5/10
1563/1563 [==============================] - 10s 7ms/step - loss: 1.4319 - accuracy: 0.4961
Epoch 6/10
1563/1563 [==============================] - 10s 7ms/step - loss: 1.3919 - accuracy: 0.5113
Epoch 7/10
1563/1563 [==============================] - 10s 7ms/step - loss: 1.3544 - accuracy: 0.5225
Epoch 8/10
1563/1563 [==============================] - 10s 7ms/step - loss: 1.3181 - accuracy: 0.5374
Epoch 9/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.2858 - accuracy: 0.5489
Epoch 10/10
1563/1563 [==============================] - 10s 7ms/step - l